<a href="https://colab.research.google.com/github/ThisIsFarhan/Language_Models/blob/main/4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain>=0.1.17 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55723569 sha256=1d9d381385940f962c4ba9d7f50c9b59283361912d5e5989c055738fe774b14d
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-01-29 20:01:26--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1738184486&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODE4NDQ4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

In [3]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

#Chains

In [4]:
from langchain import PromptTemplate

In [5]:
#Single chain


template = """<s><|User|>
{input_prompt}<|END|>
<|Assistant|>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

basic_chain = prompt | llm
basic_chain.invoke({
    "input_prompt":"Tell me about computer engineering"
})

"Computer Engineering is an interdisciplinary branch of engineering that integrates principles from electrical engineering and computer science. It's focused on the design, development, testing, and manufacturing of both hardware (physical parts) and software for computers, servers, storage devices, mobile phones, tablets, embedded systems in appliances and vehicles etc.\n\nComputer engineers work to develop new technology that makes existing devices faster or smaller but more powerful, as well as developing entirely new technologies like quantum computing. They also focus on improving the security of these technologies and designing efficient algorithms for processing data.\n\nThe field is continually evolving due to rapid advancements in digital technology. Computer engineering requires a solid understanding of both hardware and software elements, so many professionals hold degrees in electrical or computer science. \n\nKey sub-fields within computer engineering include:\n\n1. Embedd

In [8]:
#Multiple chain
from langchain import LLMChain

#--------------------Templates------------------------------
summary_template = """<s><|user|>
Create a title for a story about {summary}. Only return the one-liner title.<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(template=summary_template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=summary_prompt, output_key="title")

title_template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(
    template=title_template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=title_prompt, output_key="character")

story_template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=story_template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")


#Defining the chain
llm_chain = title | character | story


#invokation
llm_chain.invoke("A person developing an efficient AI agent")

{'summary': 'A person developing an efficient AI agent',
 'title': ' "Crafting Intelligence: The Odyssey of Building an Efficient AI Agent"',
 'character': " The protagonist, Dr. Amelia Chen, is a brilliant and driven computer scientist who tirelessly works to develop an advanced artificial intelligence capable of learning and adapting with unprecedented efficiency. With unwavering determination and innovative problem-solving skills, she navigates the challenges of AI design while grappling with ethical concerns surrounding her creation's potential impact on society.\n\nDr. Amelia Chen is a compassionate yet fiercely independent researcher who brings together an interdisciplinary team to tackle complex problems in artificial intelligence, striving for balance between technological advancements and responsible use of technology. Her journey unfolds as she discovers the immense power her efficient AI agent holds, forcing her to confront difficult choices about how to wield this newfound 